 # Super-dataset Preprocessing

 Creates one dataframe per spli that contains *all* engineered

 – baseline features **plus** delta-coordinates, Cartesian xyz, speed deltas.

In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
from pathlib import Path

data_splits_folder = "data_splits/"

train_df = pd.read_csv(data_splits_folder + "train.csv", parse_dates=["timestamp"])
val_df = pd.read_csv(data_splits_folder + "val.csv", parse_dates=["timestamp"])
test_df = pd.read_csv(data_splits_folder + "test.csv", parse_dates=["timestamp"])

In [24]:
def dataset_details(df: pd.DataFrame) -> None:
    print(f"Number of unique aircraft: {df['flight_id'].nunique()}")
    print(f"Data length: {len(df)}")
    print(f"Data length by 30s windows: {len(df) // 30}")
    print(f"Data length by 60s windows: {len(df) // 60}")
    print(f"Data length by 120s windows: {len(df) // 120}")

    print("\n" + 15 * "-")
    print("Dataset info:")
    print(df.info(show_counts=True))

    print("\n" + 15 * "-")
    print("Dataset na percentages:")
    na_pct = df.isna().mean().sort_values(ascending=False) * 100
    display(na_pct)
    print("Any na values in dataset:", df.isna().values.any())

    print("\n" + 15 * "-")
    print(f"Unique flight ids: {df['flight_id'].nunique()}")

    print("\n" + 15 * "-")
    dupes = df.duplicated(subset=["flight_id", "timestamp"])
    print("Exact duplicate rows:", dupes.sum())

    print("\n" + 15 * "-")
    print(
        f"Number of 7700 squawks: {df['is_7700'].sum()}, percent: {df['is_7700'].sum()/len(df)*100:.2f}%"
    )
    print(
        f"Number of flights that have the squawk: {df[df['is_7700']].flight_id.nunique()}, percent: {df[df['is_7700']].flight_id.nunique()/df['flight_id'].nunique()*100:.2f}%"
    )


dataset_details(train_df)

Number of unique aircraft: 643
Data length: 3337062
Data length by 30s windows: 111235
Data length by 60s windows: 55617
Data length by 120s windows: 27808

---------------
Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3337062 entries, 0 to 3337061
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype              
---  ------         --------------    -----              
 0   timestamp      3337062 non-null  datetime64[ns, UTC]
 1   altitude       3337062 non-null  float64            
 2   flight_id      3337062 non-null  object             
 3   groundspeed    3337062 non-null  float64            
 4   latitude       3337062 non-null  float64            
 5   longitude      3337062 non-null  float64            
 6   track          3337062 non-null  float64            
 7   vertical_rate  3337062 non-null  float64            
 8   is_7700        3337062 non-null  bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(6), object(1)


timestamp        0.0
altitude         0.0
flight_id        0.0
groundspeed      0.0
latitude         0.0
longitude        0.0
track            0.0
vertical_rate    0.0
is_7700          0.0
dtype: float64

Any na values in dataset: False

---------------
Unique flight ids: 643

---------------
Exact duplicate rows: 0

---------------
Number of 7700 squawks: 968948, percent: 29.04%
Number of flights that have the squawk: 643, percent: 100.00%


In [25]:
dataset_details(test_df)

Number of unique aircraft: 80
Data length: 405068
Data length by 30s windows: 13502
Data length by 60s windows: 6751
Data length by 120s windows: 3375

---------------
Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405068 entries, 0 to 405067
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   timestamp      405068 non-null  datetime64[ns, UTC]
 1   altitude       405068 non-null  float64            
 2   flight_id      405068 non-null  object             
 3   groundspeed    405068 non-null  float64            
 4   latitude       405068 non-null  float64            
 5   longitude      405068 non-null  float64            
 6   track          405068 non-null  float64            
 7   vertical_rate  405068 non-null  float64            
 8   is_7700        405068 non-null  bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(6), object(1)
memory usage: 25.1

timestamp        0.0
altitude         0.0
flight_id        0.0
groundspeed      0.0
latitude         0.0
longitude        0.0
track            0.0
vertical_rate    0.0
is_7700          0.0
dtype: float64

Any na values in dataset: False

---------------
Unique flight ids: 80

---------------
Exact duplicate rows: 0

---------------
Number of 7700 squawks: 125526, percent: 30.99%
Number of flights that have the squawk: 80, percent: 100.00%


In [26]:
dataset_details(val_df)

Number of unique aircraft: 80
Data length: 432892
Data length by 30s windows: 14429
Data length by 60s windows: 7214
Data length by 120s windows: 3607

---------------
Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432892 entries, 0 to 432891
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   timestamp      432892 non-null  datetime64[ns, UTC]
 1   altitude       432892 non-null  float64            
 2   flight_id      432892 non-null  object             
 3   groundspeed    432892 non-null  float64            
 4   latitude       432892 non-null  float64            
 5   longitude      432892 non-null  float64            
 6   track          432892 non-null  float64            
 7   vertical_rate  432892 non-null  float64            
 8   is_7700        432892 non-null  bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(6), object(1)
memory usage: 26.8

timestamp        0.0
altitude         0.0
flight_id        0.0
groundspeed      0.0
latitude         0.0
longitude        0.0
track            0.0
vertical_rate    0.0
is_7700          0.0
dtype: float64

Any na values in dataset: False

---------------
Unique flight ids: 80

---------------
Exact duplicate rows: 0

---------------
Number of 7700 squawks: 120291, percent: 27.79%
Number of flights that have the squawk: 80, percent: 100.00%


In [27]:
train_df.head()

,timestamp,altitude,flight_id,groundspeed,latitude,longitude,track,vertical_rate,is_7700
0,2018-01-01 12:29:49+00:00,11300.0,ARG1511_20180101,309.885195,-31.449051,-63.963318,130.441641,3008.000000,True
1,2018-01-01 12:29:50+00:00,11375.0,ARG1511_20180101,310.264709,-31.450453,-63.961432,130.441641,3040.000000,True
2,2018-01-01 12:29:51+00:00,11450.0,ARG1511_20180101,310.644223,-31.451856,-63.959546,130.441641,3072.000000,True
3,2018-01-01 12:29:52+00:00,11500.0,ARG1511_20180101,311.192793,-31.452843,-63.958193,130.440361,3074.742857,True
4,2018-01-01 12:29:53+00:00,11550.0,ARG1511_20180101,311.741363,-31.453831,-63.956840,130.439081,3077.485714,True


In [28]:
# The dataset is cleaned of NaNs but we can use the following as future proof
# num_cols_to_fill = ["groundspeed", "track", "vertical_rate"]
# for _df in (train_df, val_df, test_df):
#     _df.sort_values("timestamp", inplace=True)
#     _df[num_cols_to_fill] = (_df.set_index("timestamp")[num_cols_to_fill]
#                                .interpolate("time")
#                                .fillna(method="ffill")
#                                .reset_index(drop=True))

 ## 1. Angle encoding -> sin/cos for latitude, longitude, track

 Removes the +-180* wrap-around discontinuity.

In [29]:
def add_angle_features(df, col_deg, prefix):
    rad = np.deg2rad(df[col_deg])
    df[f"{prefix}_sin"] = np.sin(rad)
    df[f"{prefix}_cos"] = np.cos(rad)
    df.drop(columns=[col_deg], inplace=True)


for _df in (train_df, val_df, test_df):
    add_angle_features(_df, "latitude", "lat")
    add_angle_features(_df, "longitude", "lon")
    add_angle_features(_df, "track", "trk")

 ## 2. Turn-rate

 Quick first derivative of heading per flight.

In [30]:
for _df in (train_df, val_df, test_df):
    _df["turn_rate"] = _df.groupby("flight_id")["trk_cos"].diff().fillna(0)

In [31]:
train_df.head()

,timestamp,altitude,flight_id,groundspeed,vertical_rate,is_7700,lat_sin,lat_cos,lon_sin,lon_cos,trk_sin,trk_cos,turn_rate
0,2018-01-01 12:29:49+00:00,11300.0,ARG1511_20180101,309.885195,3008.000000,True,-0.521740,0.853104,-0.898513,0.438946,0.761067,-0.648673,0.000000
1,2018-01-01 12:29:50+00:00,11375.0,ARG1511_20180101,310.264709,3040.000000,True,-0.521761,0.853092,-0.898499,0.438976,0.761067,-0.648673,0.000000
2,2018-01-01 12:29:51+00:00,11450.0,ARG1511_20180101,310.644223,3072.000000,True,-0.521782,0.853079,-0.898484,0.439006,0.761067,-0.648673,0.000000
3,2018-01-01 12:29:52+00:00,11500.0,ARG1511_20180101,311.192793,3074.742857,True,-0.521797,0.853070,-0.898474,0.439027,0.761082,-0.648656,0.000017
4,2018-01-01 12:29:53+00:00,11550.0,ARG1511_20180101,311.741363,3077.485714,True,-0.521811,0.853061,-0.898464,0.439048,0.761096,-0.648639,0.000017


Until now we have the same features as in the baseline preprocessed dataset.

 ## 3. Per-flight delta-coordinates

 Centred on the first row of each flight – useful for GRU-delta experiments.

In [32]:
def add_deltas(df: pd.DataFrame) -> pd.DataFrame:
    # first-row reference table
    ref = (
        df.groupby("flight_id")
        .first()[["altitude", "lat_sin", "lat_cos", "lon_sin", "lon_cos"]]
        .rename(columns=lambda c: f"{c}_0")
    )

    df = df.merge(ref, left_on="flight_id", right_index=True, how="left")

    # deltas
    df["d_alt"] = df["altitude"] - df["altitude_0"]

    lat = np.rad2deg(np.arctan2(df["lat_sin"], df["lat_cos"]))
    lon = np.rad2deg(np.arctan2(df["lon_sin"], df["lon_cos"]))
    lat0 = np.rad2deg(np.arctan2(df["lat_sin_0"], df["lat_cos_0"]))
    lon0 = np.rad2deg(np.arctan2(df["lon_sin_0"], df["lon_cos_0"]))

    df["d_lat"] = lat - lat0
    df["d_lon"] = lon - lon0

    df.drop(columns=[c for c in df.columns if c.endswith(("_0", "_deg"))], inplace=True)
    return df


train_df = add_deltas(train_df)
val_df = add_deltas(val_df)
test_df = add_deltas(test_df)

In [33]:
train_df.head()

,timestamp,altitude,flight_id,groundspeed,vertical_rate,is_7700,lat_sin,lat_cos,lon_sin,lon_cos,trk_sin,trk_cos,turn_rate,d_alt,d_lat,d_lon
0,2018-01-01 12:29:49+00:00,11300.0,ARG1511_20180101,309.885195,3008.000000,True,-0.521740,0.853104,-0.898513,0.438946,0.761067,-0.648673,0.000000,0.0,0.000000,0.000000
1,2018-01-01 12:29:50+00:00,11375.0,ARG1511_20180101,310.264709,3040.000000,True,-0.521761,0.853092,-0.898499,0.438976,0.761067,-0.648673,0.000000,75.0,-0.001402,0.001886
2,2018-01-01 12:29:51+00:00,11450.0,ARG1511_20180101,310.644223,3072.000000,True,-0.521782,0.853079,-0.898484,0.439006,0.761067,-0.648673,0.000000,150.0,-0.002805,0.003772
3,2018-01-01 12:29:52+00:00,11500.0,ARG1511_20180101,311.192793,3074.742857,True,-0.521797,0.853070,-0.898474,0.439027,0.761082,-0.648656,0.000017,200.0,-0.003792,0.005125
4,2018-01-01 12:29:53+00:00,11550.0,ARG1511_20180101,311.741363,3077.485714,True,-0.521811,0.853061,-0.898464,0.439048,0.761096,-0.648639,0.000017,250.0,-0.004780,0.006478


 ## 4. Earth-centred Cartesian XYZ

 Useful for Transformer dot-product geometry.

In [ ]:
R_EARTH = 6_371_000.0  # metres


def add_xyz(df: pd.DataFrame) -> None:
    lat = np.arctan2(df["lat_sin"], df["lat_cos"])
    lon = np.arctan2(df["lon_sin"], df["lon_cos"])
    h = df["altitude"]  # already numeric

    df["x"] = (R_EARTH + h) * np.cos(lat) * np.cos(lon)
    df["y"] = (R_EARTH + h) * np.cos(lat) * np.sin(lon)
    df["z"] = (R_EARTH + h) * np.sin(lat)


for _df in (train_df, val_df, test_df):
    add_xyz(_df)

 ## 5. Dynamic feature – delta ground-speed

In [35]:
for _df in (train_df, val_df, test_df):
    _df["d_groundspeed"] = _df.groupby("flight_id")["groundspeed"].diff().fillna(0)

 ## 6. StandardScaler

In [36]:
NUMERIC_COLS = [
    "altitude",
    "groundspeed",
    "vertical_rate",
    "lat_sin",
    "lat_cos",
    "lon_sin",
    "lon_cos",
    "trk_sin",
    "trk_cos",
    "turn_rate",
    "d_alt",
    "d_lat",
    "d_lon",
    "d_groundspeed",
    "x",
    "y",
    "z",
]

scaler = StandardScaler()
train_df[NUMERIC_COLS] = scaler.fit_transform(train_df[NUMERIC_COLS])
val_df[NUMERIC_COLS] = scaler.transform(val_df[NUMERIC_COLS])
test_df[NUMERIC_COLS] = scaler.transform(test_df[NUMERIC_COLS])

In [39]:
train_df.head()

,timestamp,altitude,flight_id,groundspeed,vertical_rate,is_7700,lat_sin,lat_cos,lon_sin,lon_cos,trk_sin,trk_cos,turn_rate,d_alt,d_lat,d_lon,x,y,z,d_groundspeed
0,2018-01-01 12:29:49+00:00,-1.272230,ARG1511_20180101,-0.820034,2.942125,True,-5.302007,1.133218,-0.790238,0.063286,1.170042,-1.137935,0.000706,-0.993039,-0.214610,0.108718,0.279288,-0.896068,-5.297662,0.015997
1,2018-01-01 12:29:50+00:00,-1.266013,ARG1511_20180101,-0.816310,2.973086,True,-5.302103,1.133093,-0.790213,0.063339,1.170042,-1.137935,0.000706,-0.988476,-0.214897,0.108906,0.279350,-0.896035,-5.297786,0.761563
2,2018-01-01 12:29:51+00:00,-1.259797,ARG1511_20180101,-0.812586,3.004047,True,-5.302199,1.132967,-0.790187,0.063392,1.170042,-1.137935,0.000706,-0.983913,-0.215184,0.109093,0.279412,-0.896002,-5.297910,0.761563
3,2018-01-01 12:29:52+00:00,-1.255652,ARG1511_20180101,-0.807202,3.006700,True,-5.302266,1.132879,-0.790169,0.063429,1.170062,-1.137908,0.003614,-0.980871,-0.215386,0.109227,0.279456,-0.895978,-5.297996,1.093678
4,2018-01-01 12:29:53+00:00,-1.251507,ARG1511_20180101,-0.801819,3.009354,True,-5.302334,1.132791,-0.790150,0.063467,1.170081,-1.137882,0.003614,-0.977829,-0.215588,0.109361,0.279499,-0.895954,-5.298082,1.093678


 ## 7. Feature-set dictionary

 To Maps *model-key* > column list to slice before windowing

In [37]:
FEATURE_SETS = {
    "baseline": [
        "altitude",
        "groundspeed",
        "vertical_rate",
        "lat_sin",
        "lat_cos",
        "lon_sin",
        "lon_cos",
        "trk_sin",
        "trk_cos",
        "turn_rate",
    ],
    "gru_delta": [
        "altitude",
        "groundspeed",
        "vertical_rate",
        "d_alt",
        "d_lat",
        "d_lon",
        "trk_sin",
        "trk_cos",
        "turn_rate",
        "d_groundspeed",
    ],
    "transformer_xyz": ["x", "y", "z", "groundspeed", "vertical_rate", "turn_rate"],
}

### Save super-payload

In [ ]:
OUT_DIR = Path("optimized_data_processing")
OUT_DIR.mkdir(exist_ok=True)

payload = {
    "train_df": train_df,
    "val_df": val_df,
    "test_df": test_df,
    "scaler": scaler,
    "numeric_cols": NUMERIC_COLS,
    "feature_sets": FEATURE_SETS,
}
joblib.dump(payload, OUT_DIR / "super_data_payload.pkl")
joblib.dump(scaler, OUT_DIR / "super_scaler.pkl")
print(f"Super-dataset saved to {OUT_DIR}/super_data_payload.pkl")

Super-dataset saved to optimized_data_processing/super_data_payload.pkl
